<a href="https://colab.research.google.com/github/CAEG10/MaskRCNN_proyecto/blob/main/mosca_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings

warnings.filterwarnings('ignore',category=FutureWarning)



from mrcnn.config import Config

from mrcnn import model as modellib

from mrcnn import visualize

import numpy as np

import colorsys

import argparse

import imutils

import random

import cv2

import os

from matplotlib import pyplot as plt



weights='mask_rcnn_mosca_0010.h5'

labels='labels.txt'

image='images/2.JPG'



#weights='mask_rcnn_coco.h5'

#labels='coco_labels.txt'

#image='nestor.jpg'







###############################[DETECCION DE PROFUNDIDAD 3D

imgL = cv2.imread('images/img_L.png',0)

imgR = cv2.imread('images/img_R.png',0)

stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)

disparity = stereo.compute(imgL,imgR)

cv2.imwrite('cv_disp_orig.jpg', disparity)

#plt.imshow(disparity,'gray')

#plt.show()

############################################################







CLASS_NAMES = open(labels).read().strip().split("\n")

#print(CLASS_NAMES)



####################################################################

hsv = [(i / len(CLASS_NAMES), 1, 1.0) for i in range(len(CLASS_NAMES))]

COLORS = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))

random.seed(42)

random.shuffle(COLORS)



####################################################################

class SimpleConfig(Config):

 NAME = "coco_inference"

 GPU_COUNT = 1

 IMAGES_PER_GPU = 1

 NUM_CLASSES = len(CLASS_NAMES)



config = SimpleConfig()

print("[INFO] loading Mask R-CNN model...")



model = modellib.MaskRCNN(mode="inference", config=config ,model_dir=os.getcwd())

model.load_weights(weights, by_name=True)



image = cv2.imread(image)

cv2.imwrite("original.jpg", image)

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

image = imutils.resize(image, width=1024,height=780)

#dim=(1024,780)

#image=cv2.resize(image,dim)



disparity = imutils.resize(disparity, width=1024)



print("[INFO] making predictions with Mask R-CNN...")

r = model.detect([image], verbose=1)[0]



for i in range(0, r["rois"].shape[0]):

 classID = r["class_ids"][i]

 mask = r["masks"][:, :, i]

 color = COLORS[classID][::-1]



 #SE APLICA MASCARA A LA IMAGEN



 image = visualize.apply_mask(image, mask, color, alpha=0.5)



# convert the image back to BGR so we can use OpenCV's drawing

# functions

image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)



#####################################################

f=open("coord.txt","a")



image3 = cv2.imread('original.jpg',1)

for i in range(0, len(r["scores"])):



 (startY, startX, endY, endX) = r["rois"][i]

 classID = r["class_ids"][i]

 label = CLASS_NAMES[classID]

 score = r["scores"][i]

 color = [int(c) for c in np.array(COLORS[classID]) * 255]



 #RECTANGULOS

 cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)



 ###################################

 #RECORTA ROI Y EXPORTA A ARCHIVO

 x=startX

 w=abs(endX-startX)

 y=startY

 h=abs(endY-startY)



 crop_img = image3[y:y+h, x:x+w]

 cv2.imshow("cropped"+str(i), crop_img)

 cv2.imwrite("cropped"+str(i)+".jpg", crop_img)